### Setup of the env

Install and import of python libraries 

In [9]:
!pip3 install selenium
!pip3 install pandas

In [10]:
import requests
import pprint
import pandas as pd
import time
from selenium import webdriver as wd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import selenium
import json

Set the following variables to download data:
- sleep_time: to be polite with TEDx (number of seconds between different request)

In [11]:
sleep_time = 0.1
log = True

This notebook uses Chrome Driver to simulate user interaction with TEDx.
To set up Chrome Driver on your laptop please refer to https://chromedriver.chromium.org/downloads


In [12]:
chromedriver_path =  '/Users/feder/Downloads/chromedriver/chromedriver'

In [28]:
def get_browser():
    chrome_options = wd.ChromeOptions()
    chrome_options.add_argument('log-level=3')
    browser = wd.Chrome(chromedriver_path, options=chrome_options)
    return browser

browser = get_browser()

# Import TEDx data


In [29]:
df = pd.read_csv("tedx_dataset.csv") 
my_tedx_list = df.to_dict('records')
len(my_tedx_list)

4467

In [30]:
skip_sentences = ['(Applause)', '(Laughter)', '(Laughs)', '(Inaudible)']

def get_transcript(my_tedx):
    if log:
        print("Current url: " + my_tedx['url'])
    
    try:
        browser.get(my_tedx['url'] + '/transcript')
        if browser.title == "TED | 404: Not Found":
            raise Exception('Transcript not available')
        
        # ensures all timestamps are generated
        player = WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, "video[src]")))
    
        # ensures English transcript is selected
        browser.find_element_by_xpath("//select[@name='transcript']/option[@value='en']").click()
    
        l = browser.find_elements_by_css_selector(".Grid.Grid--with-gutter.d\:f\@md.p-b\:4")
    
        transcript = []
        for rel in l:
            sentence = rel.find_elements_by_css_selector("p")[0].text
            # skips useless lines
            if sentence.strip() not in skip_sentences:  
                timestamp = rel.find_elements_by_css_selector("button.sb")[0].text
                transcript.append({"timestamp": timestamp, "sentence": sentence})
        my_tedx['transcript'] = transcript
    except Exception as err:
        print(err)
        my_tedx['transcript'] = {}

    return my_tedx

In [31]:
my_tedx_list_final = []
for my_tedx in my_tedx_list:
    my_tedx_list_final.append(get_transcript(my_tedx))
    
print("Done")

Current url: https://www.ted.com/talks/alexandra_auer_the_intangible_effects_of_walls_apr_2020
Current url: https://www.ted.com/talks/elizabeth_gilbert_it_s_ok_to_feel_overwhelmed_here_s_what_to_do_next
Current url: https://www.ted.com/talks/butterscotch_accept_who_i_am
Transcript not available
Current url: https://www.ted.com/talks/ethan_lisi_what_it_s_really_like_to_have_autism
Current url: https://www.ted.com/talks/daniel_finkel_can_you_solve_the_sea_monster_riddle
Current url: https://www.ted.com/talks/matt_walker_why_sleep_matters_now_more_than_ever
Transcript not available
Current url: https://www.ted.com/talks/sonia_shah_how_to_make_pandemics_optional_not_inevitable
Transcript not available
Current url: https://www.ted.com/talks/mary_ellen_hannibal_how_you_can_help_save_the_monarch_butterfly_and_the_planet
Current url: https://www.ted.com/talks/todd_dufresne_history_vs_sigmund_freud
Current url: https://www.ted.com/talks/rabbi_lord_jonathan_sacks_how_we_can_navigate_the_coronavi

KeyboardInterrupt: 

In [32]:
transcript_dataset = []
for o in my_tedx_list_final:
    for t in o['transcript']:
        transcript_dataset.append({"idx": o['idx'], "timestamp": t["timestamp"], "sentence": t["sentence"]})

In [33]:
transcript_df = pd.DataFrame.from_dict(transcript_dataset)
transcript_df.to_csv('transcript_dataset.csv', index=False)